## Baggin

In [45]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score


In [5]:
defa=pd.read_csv("C:/Users/munoz/Downloads/lab_apre_est/Default.csv")
defa.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880


In [25]:
stu=(defa["student"]=="Yes").astype(int)
x1=defa["balance"].values.reshape(-1,1)
x3=stu.values.reshape(-1,1)
x2=defa["income"].values.reshape(-1,1)
y=defa["default"]== "Yes"
# Matriz de predictores
X3=np.hstack([x1,x2,x3])

modelo=LogisticRegression()
modelo.fit(X3,y)

#Intercepto y coeficientes
beta0_3=modelo.intercept_
beta1_3=modelo.coef_

print("Intercept: ",beta0_3)
print("Coeficientes: ",beta1_3)

defa['student_Yes'] = pd.get_dummies(defa['student'], drop_first=True)
X3 = defa[['balance', 'income', 'student_Yes']]
y= defa['default']=='Yes'
X_multi = LogisticRegression()
X_multi.fit(X3, y)

y_pred = modelo.predict(X3)
acc1 = accuracy_score(y, y_pred)
print("Accuracy del modelo (versión 1):", acc1)

Intercept:  [-10.90178652]
Coeficientes:  [[ 5.73059385e-03  3.96180616e-06 -6.12575695e-01]]
Accuracy del modelo (versión 1): 0.9732


C:\Users\munoz\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [19]:
X3 = X3.astype(float)
y = (defa['default'] == 'Yes').astype(int)

n_boot = 500
boot_scores = []

for i in range(n_boot):
    # Crear muestra bootstrap
    idx = np.random.choice(len(X3), size=len(X3), replace=True)
    X_boot = X3.iloc[idx]
    y_boot = y.iloc[idx]

    # Entrenar árbol
    tree = DecisionTreeClassifier()
    tree.fit(X_boot, y_boot)

    # Predicción fuera de bolsa (OOB)
    # puntos no usados en bootstrap
    oob_idx = np.setdiff1d(np.arange(len(X3)), idx)

    if len(oob_idx) > 0:
        X_oob = X3.iloc[oob_idx]
        y_oob = y.iloc[oob_idx]
        pred = tree.predict(X_oob)
        boot_scores.append(accuracy_score(y_oob, pred))

print("Accuracy promedio OOB bootstrap (500 árboles):", np.mean(boot_scores))


Accuracy promedio OOB bootstrap (500 árboles): 0.9556131831368643


In [49]:
n_boot = 500
n = X3.shape[0]

resultados_auc = []
resultados_accuracy = []

arbol= DecisionTreeClassifier(random_state=42)

for i in range(n_boot):

    x_res, y_res=resample(X3,y,replace=True, random_state=i)
    x_columns=np.random.choice(X3.columns,2,replace=False)
    x_res=x_res[x_columns]
    arbol.fit(x_res,y_res)

    auc= roc_auc_score(y,arbol.predict(X3[x_columns]))
    resultados_auc.append(auc)
    accuracy=accuracy_score(y,arbol.predict(X3[x_columns]))
    acc_oob_bag.append(accuracy)




print("Accuracy árbol (OOB) promedio: ", np.mean(resultados_auc))
print("Desviación estándar árbol: ", std(acc_oob_tree))



print("Moda del accuracy OOB (árbol):", moda_tree)
print("Moda del accuracy OOB (bagging 2 árboles):", moda_bag)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'